In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
tree = ET.parse('Q22020_35460_20220316.xml')
root = tree.getroot()
x = list(root[0])

In [6]:
x[1].text

'15670'

In [10]:
x[0]

<Element 'RECDATE' at 0x000001A4E2E15950>

In [3]:
data = []

# [i.tag for i in x[2].getchildren()[0].getchildren()]
# [i.tag for i in x[2][0]]
col = ['START','END','SURVEY_COUNT']+[i.tag for i in list(list(x[2])[0])]
df = pd.DataFrame([],columns=col)
s_count = x[1].text
s_date = x[0][0].text
e_date = x[0][1].text

for child in x[2]:
    info = []
    for c in child:
        col.append(c.tag)
        info.append(c.text)
    df.loc[len(df)] = [s_date,e_date,s_count] + info

In [4]:
df

,START,END,SURVEY_COUNT,SERVICE,VARNAME,QUESTION_TEXT,SECTION,STANDARD,SCREENING,TOP_BOX_SCALE,TOP_BOX_ANSWER
0,2020-04-01,2020-06-30,15670,AS,ADJSAMP,Adjusted Sample,Background,None,None,None,None
1,2020-04-01,2020-06-30,15670,AS,AGE,Age,Background,None,None,None,None
2,2020-04-01,2020-06-30,15670,AS,AGERG,Age,Background,None,None,None,None
3,2020-04-01,2020-06-30,15670,AS,ATAQSANS,Number of ATA questions answered,Background,None,None,None,None
4,2020-04-01,2020-06-30,15670,AS,BARCODE,Barcode,None,N,N,None,None
5,2020-04-01,2020-06-30,15670,AS,C185,Information the nurses gave you as they helped...,Nurses,Y,N,None,None
6,2020-04-01,2020-06-30,15670,AS,C186,Nurses' response to your concerns or questions,Nurses,Y,N,None,None
7,2020-04-01,2020-06-30,15670,AS,C3,Nurses' concern for your comfort as you recove...,Nurses,Y,N,None,None
8,2020-04-01,2020-06-30,15670,AS,CMPTNAME,Patient Name (Comments),Comments,N,N,None,None
9,2020-04-01,2020-06-30,15670,AS,CMPTPHON,Patient Phone Number (Comments),Comments,N,N,None,None


In [5]:
df.to_csv('header.csv')

# PATIENTLEVELDATA

In [6]:
def Get_PatientLevelData(patient):
    
    analysis_varname = []
    analysis_value = []
    for i in patient[5]:
        for j in i:
            if j.tag == 'VARNAME':
                analysis_varname.append(j.text)
            else:
                analysis_value.append(j.text)

    demographics_varname = []
    demographics_value = []
    for i in patient[6]:
        for j in i:
            if j.tag == 'VARNAME':
                demographics_varname.append(j.text)
            else:
                demographics_value.append(j.text)

    info = []
    col = []
    for i in patient[:5]:
        info.append(i.text)
        col.append(i.tag)

    comment_varname = []
    comment_sentiment = []
    comment_value = []

    if [i.tag for i in patient if i.tag=='COMMENTS']:
        for i in patient[8]:
            for j in i:
                if j.tag == 'VARNAME':
                    comment_varname.append(j.text)
                elif j.tag == 'SENTIMENT':
                    comment_sentiment.append(j.text)
                else:
                    comment_value.append(j.text)

    chaps_varname = []
    chaps_value = []
    for i in patient[7]:
        for j in i:
            if j.tag == 'VARNAME':
                chaps_varname.append(j.text)
            else:
                chaps_value.append(j.text)

    get_max = max(len(analysis_varname),len(demographics_varname),len(comment_varname),len(chaps_varname))

    col = col + ['analysis_varname','analysis_value','demographics_varname','demographics_value','comment_varname',
    'comment_sentiment','comment_value','chaps_varname','chaps_value']

    info.append(analysis_varname)
    info.append(analysis_value)
    info.append(demographics_varname)
    info.append(demographics_value)
    info.append(comment_varname)
    info.append(comment_sentiment)
    info.append(comment_value)
    info.append(chaps_varname)
    info.append(chaps_value)

    df1 = pd.DataFrame([],columns=col)

    for name,val in zip(df1,info):
        if isinstance(val,list):
            df1[name] = val + [float('NaN')]*(get_max-len(val))
        else:
    #         print(val)
            df1[name] = [val]*get_max
    return df1

In [7]:
data = []
for patient in root[1:]:
    data.append(Get_PatientLevelData(patient))
df = pd.concat(data)

In [8]:
df.to_csv('PatientLevelData.csv')

In [9]:
# df['chaps_varname'].value_counts()

In [10]:
df

,SURVEY_ID,CLIENT_ID,SERVICE,RECDATE,DISDATE,analysis_varname,analysis_value,demographics_varname,demographics_value,comment_varname,comment_sentiment,comment_value,chaps_varname,chaps_value
0,730056639,17291,OY,2020-04-03,2020-02-14,A1,4,ADMDATE,2020-02-14,NaN,NaN,NaN,NaN,NaN
1,730056639,17291,OY,2020-04-03,2020-02-14,A2,4,ADMDD,2,NaN,NaN,NaN,NaN,NaN
2,730056639,17291,OY,2020-04-03,2020-02-14,A3,4,ADMMM,1,NaN,NaN,NaN,NaN,NaN
3,730056639,17291,OY,2020-04-03,2020-02-14,A4,5,ADMYR,2020,NaN,NaN,NaN,NaN,NaN
4,730056639,17291,OY,2020-04-03,2020-02-14,B1,4,AGE,40,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2319035452,749,OU,2020-06-30,2020-06-15,NaN,NaN,LANGUAGE,English,NaN,NaN,NaN,NaN,NaN
39,2319035452,749,OU,2020-06-30,2020-06-15,NaN,NaN,PAGELINK,2319035452,NaN,NaN,NaN,NaN,NaN
40,2319035452,749,OU,2020-06-30,2020-06-15,NaN,NaN,SEX,Male,NaN,NaN,NaN,NaN,NaN
41,2319035452,749,OU,2020-06-30,2020-06-15,NaN,NaN,TIMECOMP,1249,NaN,NaN,NaN,NaN,NaN
